# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open('twits.json', 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    pattern = r'https?://.+[ $]'
    text = re.sub(pattern, ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    pattern = r'\$\w+'
    text = re.sub(pattern, ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    pattern = r'@\w+'
    text = re.sub(pattern, ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split(' ')

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(w, pos = 'v') for w in tokens if len(w)>1 ]
    tokens = [wnl.lemmatize(w, pos = 'n') for w in tokens if len(w)>1 ]
    
    
    return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\UDAYG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement

tokenized = [preprocess(message) for message in messages]

In [7]:
import numpy as np
np.array(tokenized[:10])

array([list(['great', 'buy', 'at', 'ill', 'wait']), list([]),
       list(['staanalystalert', 'for', 'jefferies', 'maintain', 'with', 'rat', 'of', 'hold', 'set', 'target', 'price', 'at', 'usd', 'our', 'own', 'verdict', 'be', 'buy', 'http', 'www', 'stocktargetadvisor', 'com', 'toprating']),
       list(['hear', 'there', 'guy', 'who', 'know', 'someone', 'who', 'think', 'somebody', 'know', 'something', 'on', 'stocktwits']),
       list(['reveal', 'yourself']),
       list(['why', 'the', 'drop', 'warren', 'buffet', 'take', 'out', 'his', 'position']),
       list(['bear', 'have', 'reason', 'on', 'to', 'pay', 'more', 'attention', 'http', 'dividendbot', 'com', 'ba']),
       list(['ok', 'good', 'we', 're', 'not', 'drop', 'in', 'price', 'over', 'the', 'weekend', 'lol']),
       list(['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above']),
       list(['drop', 'per', 'week', 'after', 'spike', 'if', 'no', 'news', 'in', 'month', 'back', 'to', 'if', 'bo', 'then', 'bingo', 'what', 'be

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [8]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
words = [w for twit in tokenized for w in twit]
bow = Counter(words)


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [9]:
# most common names
bow.most_common(15)

[('be', 593560),
 ('the', 405723),
 ('to', 394389),
 ('amp', 394031),
 ('http', 367835),
 ('com', 320532),
 ('utm', 305446),
 ('for', 281333),
 ('on', 247478),
 ('of', 217684),
 ('and', 214356),
 ('in', 212131),
 ('www', 204135),
 ('this', 204000),
 ('it', 199556)]

In [10]:
# least common names
bow.most_common()[:20:-1]

[('nrcmtghrtglsx', 1),
 ('aposbeast', 1),
 ('meatportmusic', 1),
 ('shortieturds', 1),
 ('freakjng', 1),
 ('englishman', 1),
 ('billiionairre', 1),
 ('scshaus', 1),
 ('spooktacular', 1),
 ('vvq', 1),
 ('odcw', 1),
 ('deadbefore', 1),
 ('impleme', 1),
 ('rsoadaaaqbaj', 1),
 ('whewhat', 1),
 ('daysagosome', 1),
 ('biznatch', 1),
 ('supplication', 1),
 ('valuetrader', 1),
 ('gedowngrade', 1),
 ('sqb', 1),
 ('sqg', 1),
 ('skz', 1),
 ('superhigh', 1),
 ('uptrnd', 1),
 ('curcncr', 1),
 ('crzycheapez', 1),
 ('insdrsloaded', 1),
 ('awayfrm', 1),
 ('xnm', 1),
 ('bzzzzz', 1),
 ('othiut', 1),
 ('atat', 1),
 ('haggart', 1),
 ('ashare', 1),
 ('urra', 1),
 ('stufq', 1),
 ('obgszyscti', 1),
 ('cmgo', 1),
 ('blpg', 1),
 ('fteg', 1),
 ('igcc', 1),
 ('crazytheory', 1),
 ('tmorw', 1),
 ('thestore', 1),
 ('fykbljrg', 1),
 ('otla', 1),
 ('tjio', 1),
 ('beaypl', 1),
 ('bpla', 1),
 ('cuyqrf', 1),
 ('ydbdzf', 1),
 ('jcd', 1),
 ('cvtjpa', 1),
 ('hrmmmmmm', 1),
 ('ucditbq', 1),
 ('oknep', 1),
 ('wpxenergy', 1),

In [11]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = dict(bow)

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 10

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 130000

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [w for w, count in bow.items() if count >= high_cutoff]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['at', 'for', 'of', 'be', 'http', 'www', 'com', 'on', 'the', 'to', 'in', 'and', 'this', 'it', 'utm', 'amp']


17961

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [12]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: ii for ii, word in enumerate(filtered_words, 1)} # starting id from value 1

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii: word for word, ii in vocab.items()}

# tokenized with the words not in `filtered_words` removed.
filtered = [[w for w in twit if w in vocab] for twit in tokenized]

In [13]:
filtered[:5]

[['great', 'buy', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'hold',
  'set',
  'target',
  'price',
  'usd',
  'our',
  'own',
  'verdict',
  'buy',
  'stocktargetadvisor',
  'toprating'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'stocktwits'],
 ['reveal', 'yourself']]

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [14]:
"""
To determine the keep probability, lets assume there are X number of neutral sentiments and total number of sentiments is a*X,
where a > 1.
If we remove Y neutral sentiments, then remaining neutral sentiments is X - Y and remaining total sentiments is a*X - Y
If we want around 20% neutral twits after removing Y neutral sentiments, we have this relationship:

X - Y = 0.2 * (a*X - Y)
Solving for Y,
Y = X * (5 - a) / 4

Keep probability for neutral twits is (X-Y) / X (New number of neutral twits divided by previous number of neutral twits)
X-Y = X - X * (5 - a) / 4 = X*(a-1)/4

keep_prob = (X-Y)/X = [X*(a-1)/4] / X =  (a-1)/4.
a = Total Sentiments / Neutral Sentiments, implies:

keep_prob = (Total Sentiments - Neutral Sentiments)/4/Neutral Sentiments
"""
print() # to avoid the above from printing

In [15]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [16]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1963732767557551

Finally let's convert our tokens into integer ids which we can pass to the network.

In [17]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [18]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement
        
        self.print_dims = False    # adding to assist in testing the model setup

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout = dropout, 
                           batch_first = False)
        # Dropout Layer
        self.dropout = nn.Dropout(0.3)
        # Linear and Softmax Layer
        self.fc = nn.Linear(lstm_size, output_size)
        self.logsoftmax = nn.LogSoftmax(dim = 1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        
        batch_size = nn_input.size(1)   # batch size is the second value in shape here. batch_first is false
                                        # for lstm as a result
        if self.print_dims:
            print('batch size: {}'.format(batch_size))
        
        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        if self.print_dims:
            print('embeds shape: {}'.format(embeds.shape))
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        if self.print_dims:
            print('lstm_out shape: {}'.format(lstm_out.shape))
        
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        if self.print_dims:
            print('lstm_out shape after stacking: {}'.format(lstm_out.shape))
        
        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        if self.print_dims:
            print('out.shape: {}'.format(out.shape))
        
        # Log softmax output
        logps = self.logsoftmax(out)
        if self.print_dims:
            print('logps shape: {}'.format(logps.shape))
            print(logps)
        
        # Reshape to be batch_size second(in line with input dimensions and output size as third)
        logps = logps.view(-1, batch_size, self.output_size)
        if self.print_dims:
            print('logps shape after reshaping: {}'.format(logps.shape))
            print(logps)
        logps = logps[-1,:,:] # get last label for each batch
        
        return logps, hidden_state

### View Model

In [19]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
# Added print commands for testing the model setup
print(model)
model.print_dims = True
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)
logps, _ = model.forward(input, hidden)
print('logps shape output: {}'.format(logps.shape))
print(logps)

TextClassifier(
  (embedding): Embedding(17961, 10)
  (lstm): LSTM(10, 6, num_layers=2, dropout=0.1)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=6, out_features=5, bias=True)
  (logsoftmax): LogSoftmax()
)
batch size: 4
embeds shape: torch.Size([5, 4, 10])
lstm_out shape: torch.Size([5, 4, 6])
lstm_out shape after stacking: torch.Size([20, 6])
out.shape: torch.Size([20, 5])
logps shape: torch.Size([20, 5])
tensor([[-1.8172, -1.3649, -1.5685, -1.4704, -1.9384],
        [-1.7910, -1.3752, -1.5941, -1.4651, -1.9223],
        [-1.8237, -1.3569, -1.5628, -1.4770, -1.9432],
        [-1.8476, -1.2744, -1.4864, -1.5809, -2.0341],
        [-1.8136, -1.4128, -1.6172, -1.4104, -1.8906],
        [-1.8752, -1.3734, -1.5990, -1.3991, -1.9345],
        [-1.8489, -1.3897, -1.5875, -1.4287, -1.9012],
        [-1.8354, -1.4032, -1.5953, -1.4303, -1.8802],
        [-1.7992, -1.3185, -1.4900, -1.5984, -1.9700],
        [-1.8483, -1.4136, -1.6212, -1.3962, -1.8706],
        [-1.85

## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [20]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [21]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 
split_frac = 0.8
train_len = int(split_frac * len(token_ids))
    
# Split training and validation set
train_features = token_ids[:train_len]
valid_features = token_ids[train_len:]
train_labels = sentiments[:train_len]
valid_labels = sentiments[train_len:]

#(len(token_ids) - training_len) % 64

In [22]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(17962, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (logsoftmax): LogSoftmax()
)

In [24]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 5
batch_size = 1024
learning_rate = 0.001
clips = 5

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        # initialise hidden using shape of labels so that this works for the last batch when 
        # batch size can be less than the declared size
        hidden = model.init_hidden(labels.shape[0])
        hidden = tuple([each.data for each in hidden])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        
        # zero accumulated gradients
        model.zero_grad()
        
        # get the output from the model
        logps, hidden = model(text_batch, hidden)
        
        # calculate the loss and perform backprop
        loss = criterion(logps.squeeze(), labels.long())
        loss.backward()
        
        # clip exploding gradients
        nn.utils.clip_grad_norm_(model.parameters(), clips)
        
        # run optimizer
        optimizer.step()
        
        #loss stats        
        if steps % print_every == 0:
            model.eval()
            
            # TODO Implement: Print metrics
            
            val_losses = []
            
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size=batch_size, 
                                             sequence_length=20, shuffle=True):
                # creating new variables for the hidden state, otherwise we'd backprop through the 
                # entire training history. We use shape of labels to initialise hidden state so 
                # that the model runs correctly for the last iteration of dataloader when the 
                # size of batch can be less than batch_size
                val_h = model.init_hidden(labels.shape[0])  
                
                val_h = tuple([each.data for each in val_h])
                
                inputs, labels = inputs.to(device), labels.to(device)
                
                for each in val_h:
                    each.to(device)
                
                output, val_h = model(inputs, val_h)
                
                # Get validation loss
                val_loss = criterion(output.squeeze(), labels.long())
                val_losses.append(val_loss.item())
                
                # Get validation accuracy
                ps = torch.exp(output.squeeze())
                top_p, top_class = ps.topk(1, dim = 1) # we only want the top class
                equals = top_class == labels.view(*top_class.shape)
                accuracy = torch.mean(equals.type(torch.FloatTensor))
                accuracy = accuracy.item()
            model.train()
            
            print('Epoch: {}/{}...'.format(epoch+1, epochs),
                 'Step: {}...'.format(steps),
                 'Training Loss: {:.6f}...'.format(loss.item()),
                 'Validation Loss: {:.6f}...'.format(np.mean(val_losses)),
                 'Validation Accuracy: {:.3f}...'.format(accuracy))
            

Starting epoch 1
Epoch: 1/5... Step: 100... Training Loss: 0.911343... Validation Loss: 0.931271... Validation Accuracy: 0.562...
Epoch: 1/5... Step: 200... Training Loss: 0.768994... Validation Loss: 0.845712... Validation Accuracy: 0.700...
Epoch: 1/5... Step: 300... Training Loss: 0.795446... Validation Loss: 0.796878... Validation Accuracy: 0.719...
Epoch: 1/5... Step: 400... Training Loss: 0.781714... Validation Loss: 0.776079... Validation Accuracy: 0.719...
Epoch: 1/5... Step: 500... Training Loss: 0.747552... Validation Loss: 0.761928... Validation Accuracy: 0.712...
Epoch: 1/5... Step: 600... Training Loss: 0.682463... Validation Loss: 0.741557... Validation Accuracy: 0.731...
Epoch: 1/5... Step: 700... Training Loss: 0.665967... Validation Loss: 0.734395... Validation Accuracy: 0.725...
Epoch: 1/5... Step: 800... Training Loss: 0.679778... Validation Loss: 0.729310... Validation Accuracy: 0.681...
Starting epoch 2
Epoch: 2/5... Step: 100... Training Loss: 0.693470... Validati

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [25]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [w for w in tokens if w in vocab]
    # Convert words to ids
    tokens = [vocab[w] for w in tokens]
        
    # Adding a batch dimension
    batch_size = 1
    text_input = np.array(tokens).reshape(len(tokens), batch_size)
    text_input = torch.from_numpy(text_input)
       
    # Get the NN output
    hidden = model.init_hidden(batch_size)
    logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred.detach().numpy()

In [26]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

array([[5.1847310e-05, 1.1260105e-02, 5.0746687e-03, 8.5176677e-01,
        1.3184661e-01]], dtype=float32)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

The model predicts the highest probability for label 4 which means that the sentiment is positive. The text gives a positive sentiment but it is an individual determination to say whether it is positive or very positive. Depending on how the labels were decided in the data set, I think this can introduce some uncertainty in the prediction. This is a bias in the data and model training itself cannot resolve this. 
The prediction looks fairly clear that this is not a negative sentiment and there is no uncertainty about that.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [27]:
with open('test_twits.json', 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [28]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [29]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [30]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': array([[0.0331286 , 0.06906965, 0.27287874, 0.20169877, 0.42322418]],
       dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z'}

In [31]:
next(score_stream)

{'symbol': '$AMZN',
 'score': array([[6.7328813e-08, 7.9762794e-06, 2.1896136e-05, 9.9988341e-01,
         8.6695931e-05]], dtype=float32),
 'timestamp': '2018-11-01T00:00:42Z'}

That's it. You have successfully built a model for sentiment analysis! 